In [1]:
import time
import os
import seaborn as sns
import numpy as np
import pandas as pd
import sys
from mimas.external.features_by_alphapept.load_mzml_data import load_mzml_data
from mimas.tools.spectral_file.extract_ms1_feature import process_mzml_file, extract_ms2_spectra, find_features
%load_ext autoreload
%autoreload 2
from toolsets.spectra_operations import convert_string_to_nist
import logging
from toolsets.spectra_operations import entropy_similarity_default
from toolsets.adduct_calculator import complete_adducts, complete_formula
# from toolsets.API_gets import complete_smiles
from toolsets.search import string_search, num_search
import toolsets.spectra_operations as so
from tqdm import tqdm
from toolsets.features_by_alphapept import find_features_alphapept

Compile the cython code is highly recommended!


In [2]:
data_pfp = pd.read_csv('/Users/fanzhoukong/Documents/GitHub/Libgen_data/EAD/libgen2.0_pfp.csv')
# data_hilic = pd.read_csv('/Users/fanzhoukong/Documents/GitHub/Libgen_data/EAD/libgen2.0_hilic.csv')

In [3]:
len(data_pfp['reference_InChIKey'].unique())

1314

In [129]:
data = pd.concat([data_pfp, data_hilic], axis=0)

In [130]:
normalized_peaks = []
for index, row in tqdm(data.iterrows(), total=len(data)):
    normalized_peaks.append(so.normalize_spectra(row['peaks_recalibrated_denoised']))

100%|██████████| 3482/3482 [00:01<00:00, 2441.58it/s]


In [5]:
((27*3)+16.69)/3

32.56333333333333

In [131]:
data['peaks_recalibrated_denoised']=normalized_peaks

In [126]:
from toolsets.file_io import export_library_msp

In [132]:
export_library_msp(data,'/Users/fanzhoukong/Documents/GitHub/Libgen_data/EAD/libgen2.0_library_final.txt' ,typeofmsms='peaks_recalibrated_denoised', ifcollision_energy = False)

100%|██████████| 3482/3482 [00:21<00:00, 163.85it/s]


In [117]:
data.head()

,scan_number,precursor_mz,charge,ms1_intensity_ratio,ms1_precursor_intensity,retention_time,peaks,ms1_index,query_idx,reference_Name,...,reference_adduct,reference_precursor_mz,key,comments,peaks_recalibrated,diff_raw,diff_recalibrated,peaks_recalibrated_denoised,explained_intensity,max_unassigned_peak_intensity
0,207,155.033529,1,0.862997,2085.0,0.617400,127.05482\t2.237659\n128.06293\t5.889224\n129....,186,19,PYROCATECHUIC ACID,...,[M+H]+,155.0339,GLDQAMYCGOIJDV-UHFFFAOYSA-N[M+H]+,recalibrateddenoised,127.05502\t2.237659\n128.063129\t5.889224\n129...,-0.000371,-0.000181,127.05502\t2.237659\n136.015147\t3.731481\n137...,84.65354,5.889224
1,397,271.061066,1,0.406893,2137.0,1.208467,126.046478\t11.440678\n127.057152\t26.165256\n...,288,107,Apigenin,...,[M+H]+,271.0601,KZNIFHPLKGYRTM-UHFFFAOYSA-N[M+H]+,recalibrateddenoised,126.046678\t11.440678\n127.057352\t26.165256\n...,0.000966,0.001116,126.046678\t11.440678\n127.057352\t26.165256\n...,77.55162,67.584750
2,904,289.198404,1,0.186737,1163.0,2.516600,50.014942\t5.663288\n51.022358\t16.923754\n52....,465,438,2-decyl-3-hydroxypentanedioic acid,...,[M+H]+,289.2010,KOLGVIJQEZBWIQ-UHFFFAOYSA-N[M+H]+,recalibrateddenoised,50.015168\t5.663288\n51.022584\t16.923754\n52....,-0.002596,-0.002452,50.015168\t5.663288\n51.022584\t16.923754\n52....,99.57359,3.746144
3,837,311.181980,1,0.740417,16090.0,2.289467,126.045433\t2.053669\n127.055359\t11.007667\n1...,433,402,2-decyl-3-hydroxypentanedioic acid,...,[M+Na]+,311.1829,KOLGVIJQEZBWIQ-UHFFFAOYSA-N[M+Na]+,recalibrateddenoised,126.045633\t2.053669\n127.055559\t11.007667\n1...,-0.000920,-0.000784,126.045633\t2.053669\n127.055559\t11.007667\n1...,96.09716,16.757941
4,803,303.122302,1,0.995707,645013.0,2.237450,118.04487\t1.002273\n119.05082\t2.234901\n131....,428,373,"1-(2,4-dihydroxyphenyl)-2-(3,5-dimethoxyphenyl...",...,[M+H]+,303.1227,TURHNAJJDDIVGC-UHFFFAOYSA-N[M+H]+,recalibrateddenoised,118.045073\t1.002273\n119.051022\t2.234901\n13...,-0.000398,-0.000259,118.045073\t1.002273\n119.051022\t2.234901\n13...,99.43046,2.826807
